In [2]:
import urllib.request
import json
import pandas as pd


#Se crea el objeto cliente y se le otorga el  valor de la conexión obtenida
from shopifyclient import shopify_client
client=shopify_client()

In [3]:
import bulkquery as bq
from customers import customers 

#Se genera la solicitud en GraphQL, almacenandola en la variable Bulk

bulk_query=bq.bulk_operation(customers)

bulk=client.execute(bulk_query)


    orders{
        edges {
            node {
                id
                createdAt
                currentSubtotalLineItemsQuantity
                
    customerJourney {
          firstVisit {
            id
            source
          }
        }

                
    lineItems{
        edges {
            node {
                id
                quantity
                title
                product {
                    id
                    description
                    handle
                }
                originalUnitPriceSet {
                    shopMoney {
                        amount
                    }
                }
            }
        }
    }
 
                
    billingAddress {
                    city
                    address1
                    address2
                    country
                }

            }
        }
    }


query {
    customers{
        edges {
            node {
                id
                firstName
   

In [4]:
#Se observa el objeto bulk
bulk

'{"data":{"bulkOperationRunQuery":{"bulkOperation":{"id":"gid://shopify/BulkOperation/4019745161279","status":"CREATED"},"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}'

In [9]:
#Se recupera el status y se almacena en una variable 
status = bq.bulk_status(client,bulk)

In [10]:
#Se revisa los datos de la variable creada
status

{'data': {'node': {'id': 'gid://shopify/BulkOperation/4019745161279',
   'status': 'COMPLETED',
   'errorCode': None,
   'createdAt': '2024-10-11T00:24:15Z',
   'completedAt': '2024-10-11T00:26:17Z',
   'objectCount': '13459',
   'fileSize': '4916940',
   'url': 'https://storage.googleapis.com/shopify-tiers-assets-prod-us-east1/bulk-operation-outputs/k0p0rj1d4u20m1oxvd175ouhyuaf-final?GoogleAccessId=assets-us-prod%40shopify-tiers.iam.gserviceaccount.com&Expires=1729211177&Signature=b%2BXlx3tpv4KI%2BsWipLC6IUARfVPdWVJU5amoyeIKoxqlrxuKHmaxVWaqF5UYy0hCMCkR%2Bw2HGbvfKSCw5ielI41OxIzBW59bSVdiaDy7p6ZJsVy4FeIihDBcOLLLRxy%2BD4H6MC34sSq10WJhdfHTXuyp1iZwc3D7R8VGRg%2Fm8%2BpqvM%2BuQzj%2Fpr6Smzwr5DvESyj6WSVXPXGo2S05u5%2FqfGYFEmeLwXNKksd5pGv5hOrc7gdja%2F3Pe%2F39r9oVmm62ihjKgSi4XOmV6Ht07XppPupc52vQpTYUSSSQs4vLS21%2BHcfXxUWR9EVc0LO%2BJ0C%2F7mVAhbVu%2B%2BCpBHDJXcDlSQ%3D%3D&response-content-disposition=attachment%3B+filename%3D%22bulk-4019745161279.jsonl%22%3B+filename%2A%3DUTF-8%27%27bulk-4019745161279.

In [48]:
#Función para crear los dataframes por cada tabla
def fetch_bulk(bulk_status):
    customers = []
    orders = []
    lineitems = []
    customer_journey = []
    
    
    # Verificar si el estado de la operación es 'COMPLETED'
    if bulk_status['data']['node']['status'] == 'COMPLETED':
        url = bulk_status['data']['node']['url']
        mydata = urllib.request.urlopen(url)
        
        # Procesar las líneas de datos recibidas
        for line in mydata:
            line = json.loads(line.decode('utf8'))
            
            # Si no hay un 'parentId', se considera un cliente
            if '__parentId' not in line:
                customers.append(line)
            else:
                # Determinar si es una orden o un 'lineitem'
                child_id = line['id']
                if 'Order' in child_id:
                    orders.append(line)
                if 'LineItem' in child_id:
                    lineitems.append(line)
                if 'CustomerJourney' in child_id:
                    customer_journey.append(line)
                
        
        # Crear dataframes a partir de los registros obtenidos
        cframe = pd.DataFrame.from_records(customers)
        oframe = pd.DataFrame.from_records(orders)
        lframe = pd.DataFrame.from_records(lineitems)
        jframe = pd.DataFrame.from_records(customer_journey)
        
        # Retornar los tres dataframes
        return cframe, oframe, lframe, jframe
    
    else:
        # En caso de que la operación no esté completa, retornar None para los tres valores
        return None, None, None, None
customers, orders, lineitems, customer_journey= fetch_bulk(status)

# Verificar si los valores fueron obtenidos correctamente
if customers is not None and orders is not None and lineitems is not None and customer_journey is not None:
    print("Datos procesados exitosamente")
    # Aquí puedes seguir trabajando con los dataframes 'customers', 'orders', y 'lineitems'
else:
    print("La operación aún no se ha completado o no hay datos disponibles.")


Datos procesados exitosamente


In [49]:
lineitems

,id,quantity,title,product,originalUnitPriceSet,__parentId
0,gid://shopify/LineItem/1458992775237,1,Sábanas 300 Hilos - Bordado Silver,None,{'shopMoney': {'amount': '149.0'}},gid://shopify/Order/605504045125
1,gid://shopify/LineItem/9967951609919,1,Cubrecama Quilt Microfibra,"{'id': 'gid://shopify/Product/4505145376831', ...",{'shopMoney': {'amount': '184.0'}},gid://shopify/Order/3823615967295
2,gid://shopify/LineItem/14419652739135,1,Set de Sábanas Signature,"{'id': 'gid://shopify/Product/4693980414015', ...",{'shopMoney': {'amount': '392.0'}},gid://shopify/Order/5837952581695
3,gid://shopify/LineItem/1443611738181,4,Set de Cubrecama Quilt Spring,None,{'shopMoney': {'amount': '99.9'}},gid://shopify/Order/596940783685
4,gid://shopify/LineItem/1443611770949,1,Set de Edredón Riansares,None,{'shopMoney': {'amount': '119.9'}},gid://shopify/Order/596940783685
...,...,...,...,...,...,...
5255,gid://shopify/LineItem/14577264033855,1,Set de Sábanas Signature,"{'id': 'gid://shopify/Product/4693980414015', ...",{'shopMoney': {'amount': '632.0'}},gid://shopify/Order/5902652735551
5256,gid://shopify/LineItem/14584482267199,1,Pack Plumón + Funda Signature,"{'id': 'gid://shopify/Product/6683192623167', ...",{'shopMoney': {'amount': '642.0'}},gid://shopify/Order/5905901879359
5257,gid://shopify/LineItem/14735784476735,1,Plumón Hipoalergénico,"{'id': 'gid://shopify/Product/6624090062911', ...",{'shopMoney': {'amount': '288.0'}},gid://shopify/Order/5966086537279
5258,gid://shopify/LineItem/14844964700223,1,Set de Sábanas Luxe,"{'id': 'gid://shopify/Product/4505134596159', ...",{'shopMoney': {'amount': '413.0'}},gid://shopify/Order/6013176348735


In [16]:
orders


,id,createdAt,currentSubtotalLineItemsQuantity,customerJourney,billingAddress,__parentId
0,gid://shopify/Order/605504045125,2018-09-11T03:41:34Z,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722083086405
1,gid://shopify/Order/3823615967295,2021-06-03T00:55:17Z,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722083086405
2,gid://shopify/Order/5837952581695,2024-07-02T15:24:00Z,0,{'firstVisit': {'id': 'gid://shopify/CustomerV...,"{'city': 'Lima', 'address1': 'Laa lomas', 'add...",gid://shopify/Customer/722083086405
3,gid://shopify/Order/596940783685,2018-09-04T15:59:16Z,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722885574725
4,gid://shopify/Order/598392078405,2018-09-05T22:13:29Z,0,None,"{'city': 'Ciudad 1', 'address1': 'Jr Direccion...",gid://shopify/Customer/722986827845
...,...,...,...,...,...,...
3808,gid://shopify/Order/5858791981119,2024-07-15T18:30:04Z,1,{'firstVisit': {'id': 'gid://shopify/CustomerV...,"{'city': 'Lima', 'address1': 'Malecon Balta 78...",gid://shopify/Customer/7259410300991
3809,gid://shopify/Order/5902652735551,2024-08-08T18:29:27Z,1,None,"{'city': 'Lince', 'address1': '', 'address2': ...",gid://shopify/Customer/7344207790143
3810,gid://shopify/Order/5905901879359,2024-08-10T18:51:15Z,1,{'firstVisit': {'id': 'gid://shopify/CustomerV...,"{'city': 'Lima', 'address1': 'Av. Manuel Olgui...",gid://shopify/Customer/7346496929855
3811,gid://shopify/Order/5966086537279,2024-09-10T04:11:57Z,1,{'firstVisit': {'id': 'gid://shopify/CustomerV...,"{'city': 'Lima', 'address1': 'Av. Brasil 3120'...",gid://shopify/Customer/7399752794175


In [51]:
import pandas as pd

df_customerJourney = orders[['id', 'customerJourney']].copy()
df_customerJourney['firstVisitId'] = df_customerJourney['customerJourney'].apply(
    lambda x: x.get('firstVisit', {}).get('id') if pd.notna(x) else None
)
df_customerJourney['source'] = df_customerJourney['customerJourney'].apply(
    lambda x: x.get('firstVisit', {}).get('source') if pd.notna(x) else None
)

df_customerJourney['parentId'] = df_customerJourney['id']  # El verdadero Order ID
df_customerJourney = df_customerJourney[['firstVisitId', 'source', 'parentId']]
df_customerJourney.rename(columns={'firstVisitId': 'id'}, inplace=True)
df_customerJourney


,id,source,parentId
0,None,None,gid://shopify/Order/605504045125
1,None,None,gid://shopify/Order/3823615967295
2,gid://shopify/CustomerVisit/13755680587839,direct,gid://shopify/Order/5837952581695
3,None,None,gid://shopify/Order/596940783685
4,None,None,gid://shopify/Order/598392078405
...,...,...,...
3808,gid://shopify/CustomerVisit/13786864943167,direct,gid://shopify/Order/5858791981119
3809,None,None,gid://shopify/Order/5902652735551
3810,gid://shopify/CustomerVisit/13851033993279,Instagram,gid://shopify/Order/5905901879359
3811,gid://shopify/CustomerVisit/13926246154303,Google,gid://shopify/Order/5966086537279


In [52]:
# Verifica los nulos en firstName y lastName
customers['firstName'].fillna('Unknown', inplace=True)
customers['lastName'].fillna('Unknown', inplace=True)
customers.rename(columns={'id': 'customer_id'}, inplace=True)
customers.head()

C:\Users\sally\AppData\Local\Temp\ipykernel_16632\1566819959.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['firstName'].fillna('Unknown', inplace=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_16632\1566819959.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,customer_id,firstName,lastName
0,gid://shopify/Customer/722083086405,Rudy,Huamán
1,gid://shopify/Customer/722885574725,Maurieth,Romero
2,gid://shopify/Customer/722986827845,Josue,Huaman
3,gid://shopify/Customer/733210214469,Rudy,Huaman
4,gid://shopify/Customer/734424072261,Rosa,Arrieta


In [53]:
# Convierte 'createdAt' a datetime
orders['createdAt'] = pd.to_datetime(orders['createdAt'])
orders.rename(columns={'id': 'order_id', '__parentId': 'customer_id'}, inplace=True)
orders.head()

,order_id,createdAt,currentSubtotalLineItemsQuantity,customerJourney,billingAddress,customer_id
0,gid://shopify/Order/605504045125,2018-09-11 03:41:34+00:00,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722083086405
1,gid://shopify/Order/3823615967295,2021-06-03 00:55:17+00:00,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722083086405
2,gid://shopify/Order/5837952581695,2024-07-02 15:24:00+00:00,0,{'firstVisit': {'id': 'gid://shopify/CustomerV...,"{'city': 'Lima', 'address1': 'Laa lomas', 'add...",gid://shopify/Customer/722083086405
3,gid://shopify/Order/596940783685,2018-09-04 15:59:16+00:00,0,None,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",gid://shopify/Customer/722885574725
4,gid://shopify/Order/598392078405,2018-09-05 22:13:29+00:00,0,None,"{'city': 'Ciudad 1', 'address1': 'Jr Direccion...",gid://shopify/Customer/722986827845


In [55]:
import pandas as pd

orders['city'] = orders['billingAddress'].apply(lambda x: x.get('city') if isinstance(x, dict) else None)
orders['address1'] = orders['billingAddress'].apply(lambda x: x.get('address1') if isinstance(x, dict) else None)
orders['address2'] = orders['billingAddress'].apply(lambda x: x.get('address2') if isinstance(x, dict) else None)
orders[['billingAddress', 'city', 'address1', 'address2']].head()


,billingAddress,city,address1,address2
0,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
1,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
2,"{'city': 'Lima', 'address1': 'Laa lomas', 'add...",Lima,Laa lomas,None
3,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
4,"{'city': 'Ciudad 1', 'address1': 'Jr Direccion...",Ciudad 1,Jr Direccion 1,Referencia 1


In [57]:
lineitems['price'] = lineitems['originalUnitPriceSet'].apply(lambda x: x['shopMoney']['amount'] if pd.notna(x) else None)
lineitems['price'] = lineitems['price'].astype(float)
lineitems.rename(columns={'id': 'lineitem_id', '__parentId': 'order_id'}, inplace=True)
lineitems.head()


,lineitem_id,quantity,title,product,originalUnitPriceSet,order_id,price
0,gid://shopify/LineItem/1458992775237,1,Sábanas 300 Hilos - Bordado Silver,None,{'shopMoney': {'amount': '149.0'}},gid://shopify/Order/605504045125,149.0
1,gid://shopify/LineItem/9967951609919,1,Cubrecama Quilt Microfibra,"{'id': 'gid://shopify/Product/4505145376831', ...",{'shopMoney': {'amount': '184.0'}},gid://shopify/Order/3823615967295,184.0
2,gid://shopify/LineItem/14419652739135,1,Set de Sábanas Signature,"{'id': 'gid://shopify/Product/4693980414015', ...",{'shopMoney': {'amount': '392.0'}},gid://shopify/Order/5837952581695,392.0
3,gid://shopify/LineItem/1443611738181,4,Set de Cubrecama Quilt Spring,None,{'shopMoney': {'amount': '99.9'}},gid://shopify/Order/596940783685,99.9
4,gid://shopify/LineItem/1443611770949,1,Set de Edredón Riansares,None,{'shopMoney': {'amount': '119.9'}},gid://shopify/Order/596940783685,119.9


In [58]:
df_customerJourney.rename(columns={'id': 'firstVisit_id', 'parentId': 'order_id'}, inplace=True)
df_customerJourney.head()


,firstVisit_id,source,order_id
0,None,None,gid://shopify/Order/605504045125
1,None,None,gid://shopify/Order/3823615967295
2,gid://shopify/CustomerVisit/13755680587839,direct,gid://shopify/Order/5837952581695
3,None,None,gid://shopify/Order/596940783685
4,None,None,gid://shopify/Order/598392078405


In [59]:
df_customerJourney.info()
lineitems.info()
orders.info()
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   firstVisit_id  2749 non-null   object
 1   source         2749 non-null   object
 2   order_id       3813 non-null   object
dtypes: object(3)
memory usage: 89.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5260 entries, 0 to 5259
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lineitem_id           5260 non-null   object 
 1   quantity              5260 non-null   int64  
 2   title                 5260 non-null   object 
 3   product               2617 non-null   object 
 4   originalUnitPriceSet  5260 non-null   object 
 5   order_id              5260 non-null   object 
 6   price                 5260 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 287.8+ KB
<class 'p

In [60]:
# Revisar que todo está bien en el DataFrame de customerJourney
df_customerJourney['firstVisit_id'] = df_customerJourney['firstVisit_id'].astype('object')  # Esto debería ya estar correcto
df_customerJourney['source'] = df_customerJourney['source'].astype('object')
df_customerJourney['order_id'] = df_customerJourney['order_id'].astype('object')
df_customerJourney.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   firstVisit_id  2749 non-null   object
 1   source         2749 non-null   object
 2   order_id       3813 non-null   object
dtypes: object(3)
memory usage: 89.5+ KB


In [61]:
df_customerJourney['firstVisit_id'].fillna('Unknown', inplace=True)
df_customerJourney.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   firstVisit_id  3813 non-null   object
 1   source         2749 non-null   object
 2   order_id       3813 non-null   object
dtypes: object(3)
memory usage: 89.5+ KB


C:\Users\sally\AppData\Local\Temp\ipykernel_16632\1324825135.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customerJourney['firstVisit_id'].fillna('Unknown', inplace=True)


In [62]:
orders['city'].fillna('Unknown', inplace=True)
orders['address1'].fillna('Unknown', inplace=True)
orders['address2'].fillna('Unknown', inplace=True)
orders[['billingAddress', 'city', 'address1', 'address2']].head()


C:\Users\sally\AppData\Local\Temp\ipykernel_16632\1987937717.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders['city'].fillna('Unknown', inplace=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_16632\1987937717.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,billingAddress,city,address1,address2
0,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
1,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
2,"{'city': 'Lima', 'address1': 'Laa lomas', 'add...",Lima,Laa lomas,Unknown
3,"{'city': 'Lima', 'address1': 'Av. Las Lomas De...",Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
4,"{'city': 'Ciudad 1', 'address1': 'Jr Direccion...",Ciudad 1,Jr Direccion 1,Referencia 1


In [64]:
df_orders_clean = orders.drop(columns=['customerJourney'])
df_orders_clean = df_orders_clean.drop(columns=['billingAddress'])
df_orders_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   order_id                          3813 non-null   object             
 1   createdAt                         3813 non-null   datetime64[ns, UTC]
 2   currentSubtotalLineItemsQuantity  3813 non-null   int64              
 3   customer_id                       3813 non-null   object             
 4   city                              3813 non-null   object             
 5   address1                          3813 non-null   object             
 6   address2                          3813 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 208.7+ KB


In [65]:
lineitems['lineitem_id'] = lineitems['lineitem_id'].astype('object')
lineitems['product'] = lineitems['product'].astype('object') 
lineitems['order_id'] = lineitems['order_id'].astype('object')
lineitems.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5260 entries, 0 to 5259
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lineitem_id           5260 non-null   object 
 1   quantity              5260 non-null   int64  
 2   title                 5260 non-null   object 
 3   product               2617 non-null   object 
 4   originalUnitPriceSet  5260 non-null   object 
 5   order_id              5260 non-null   object 
 6   price                 5260 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 287.8+ KB


In [66]:
df_orders_clean['order_id'] = df_orders_clean['order_id'].astype('object')
df_orders_clean['customer_id'] = df_orders_clean['customer_id'].astype('object')
df_orders_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   order_id                          3813 non-null   object             
 1   createdAt                         3813 non-null   datetime64[ns, UTC]
 2   currentSubtotalLineItemsQuantity  3813 non-null   int64              
 3   customer_id                       3813 non-null   object             
 4   city                              3813 non-null   object             
 5   address1                          3813 non-null   object             
 6   address2                          3813 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 208.7+ KB


In [67]:
customers['customer_id'] = customers['customer_id'].astype('object')
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  4386 non-null   object
 1   firstName    4386 non-null   object
 2   lastName     4386 non-null   object
dtypes: object(3)
memory usage: 102.9+ KB


In [69]:
import pandas as pd
customers['customer_id'] = customers['customer_id'].str.extract(r'(\d+)')
customers.head()


,customer_id,firstName,lastName
0,722083086405,Rudy,Huamán
1,722885574725,Maurieth,Romero
2,722986827845,Josue,Huaman
3,733210214469,Rudy,Huaman
4,734424072261,Rosa,Arrieta


In [70]:
df_orders_clean['order_id'] = df_orders_clean['order_id'].str.extract(r'(\d+)')
df_orders_clean['customer_id'] = df_orders_clean['customer_id'].str.extract(r'(\d+)')
df_orders_clean.head()


,order_id,createdAt,currentSubtotalLineItemsQuantity,customer_id,city,address1,address2
0,605504045125,2018-09-11 03:41:34+00:00,0,722083086405,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
1,3823615967295,2021-06-03 00:55:17+00:00,0,722083086405,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
2,5837952581695,2024-07-02 15:24:00+00:00,0,722083086405,Lima,Laa lomas,Unknown
3,596940783685,2018-09-04 15:59:16+00:00,0,722885574725,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina
4,598392078405,2018-09-05 22:13:29+00:00,0,722986827845,Ciudad 1,Jr Direccion 1,Referencia 1


In [71]:
lineitems['lineitem_id'] = lineitems['lineitem_id'].str.extract(r'(\d+)')
lineitems['order_id'] = lineitems['order_id'].str.extract(r'(\d+)')
lineitems.head()


,lineitem_id,quantity,title,product,originalUnitPriceSet,order_id,price
0,1458992775237,1,Sábanas 300 Hilos - Bordado Silver,None,{'shopMoney': {'amount': '149.0'}},605504045125,149.0
1,9967951609919,1,Cubrecama Quilt Microfibra,"{'id': 'gid://shopify/Product/4505145376831', ...",{'shopMoney': {'amount': '184.0'}},3823615967295,184.0
2,14419652739135,1,Set de Sábanas Signature,"{'id': 'gid://shopify/Product/4693980414015', ...",{'shopMoney': {'amount': '392.0'}},5837952581695,392.0
3,1443611738181,4,Set de Cubrecama Quilt Spring,None,{'shopMoney': {'amount': '99.9'}},596940783685,99.9
4,1443611770949,1,Set de Edredón Riansares,None,{'shopMoney': {'amount': '119.9'}},596940783685,119.9


In [72]:
df_customerJourney['firstVisit_id'] = df_customerJourney['firstVisit_id'].str.extract(r'(\d+)')
df_customerJourney['order_id'] = df_customerJourney['order_id'].str.extract(r'(\d+)')
df_customerJourney.head()


,firstVisit_id,source,order_id
0,NaN,None,605504045125
1,NaN,None,3823615967295
2,13755680587839,direct,5837952581695
3,NaN,None,596940783685
4,NaN,None,598392078405


In [73]:
valores_unicos_source = df_customerJourney['source'].unique()
print(valores_unicos_source)


[None 'direct' 'Facebook' 'Instagram' 'Google'
 'https://www.tiendeo.cl/ofertas-catalogos/cannon-home'
 'https://googleads.g.doubleclick.net/pagead/ads'
 'https://youtube.com/mobile' 'email'
 'https://www.googleadservices.com/pagead/aclk' 'https://rpp.pe/'
 'https://larepublica.pe/amp/espectaculos/1473143-jefferson-farfan-yahaira-plasencia-jazmin-pinedo-confiesa-son-pareja-mujeres-mando-latina-video'
 'https://tpc.googlesyndication.com/safeframe/1-0-33/html/container.html'
 'https://www.linio.com.pe/search'
 'https://tpc.googlesyndication.com/safeframe/1-0-35/html/container.html'
 'https://www-americatv-com-pe.cdn.ampproject.org/v/s/www.americatv.com.pe/amp/en-boca-de-todos/flavia-laos-desplazara-sheyla-rojas-como-conductora-estas-todas-noticia-105383'
 'https://www.mercadopago.com.pe/'
 'https://www.dinersclub.pe/mothersweek/'
 'android-app://com.google.android.gm/' 'android-app://m.facebook.com/'
 'https://facebook.com/'
 'android-app://com.google.android.googlequicksearchbox/'
 'htt

In [74]:
import pandas as pd

# Función para categorizar las fuentes
def generalizar_source(source):
    if pd.isna(source) or source == 'an unknown source':
        return 'Unknown'  # Generalizar None y 'an unknown source' como 'Unknown'
    elif 'android-app' in source:
        return 'Android App'
    elif 'googleads' in source or 'googlesyndication' in source or 'pagead' in source:
        return 'Google Ads'
    elif 'youtube' in source:
        return 'YouTube'
    elif 'mercadopago' in source or 'dinersclub' in source or 'interbank' in source:
        return 'Promociones Bancarias'
    elif 'facebook' in source:
        return 'Facebook'
    elif 'linio' in source or 'tiendeo' in source:
        return 'E-Commerce'
    elif source.startswith('http') or source.startswith('https'):
        return 'Other Website'
    else:
        return source  # Mantener el valor original si no es un enlace

# Aplicar la función para generalizar la columna 'source'
df_customerJourney['source_generalizada'] = df_customerJourney['source'].apply(generalizar_source)

# Verificar el resultado
df_customerJourney[['source', 'source_generalizada']].head()


,source,source_generalizada
0,None,Unknown
1,None,Unknown
2,direct,direct
3,None,Unknown
4,None,Unknown


In [75]:
valores_unicos_source = df_customerJourney['source_generalizada'].unique()
print(valores_unicos_source)


['Unknown' 'direct' 'Facebook' 'Instagram' 'Google' 'E-Commerce'
 'Google Ads' 'YouTube' 'email' 'Other Website' 'Promociones Bancarias'
 'Android App' 'DuckDuckGo' 'Bing' 'Yahoo']


In [76]:
valores_unicos_source = lineitems['title'].unique()
print(valores_unicos_source)

['Sábanas 300 Hilos - Bordado Silver' 'Cubrecama Quilt Microfibra'
 'Set de Sábanas Signature' 'Set de Cubrecama Quilt Spring'
 'Set de Edredón Riansares' 'Sábanas de Algodón 300 Hilos - Bordado White'
 'Set de Cubrecama Quilt Royce' 'Edredón Royal - Queen'
 'Set de Edredón Azurium' 'Cubrecama 2 Plazas Royce'
 'Cubrecama 2 Plazas Peach' 'Set de Edredón Caledonia'
 'Sábanas 300 Hilos - 2 Plazas' 'Set de Edredón Galdina'
 'Edredón Azurium - 2 Plazas' 'Set de Edredón Marena'
 'Edredón 1.5 Plazas Galdina' 'Edredón Royal - 2 Plazas'
 'Set de Edredón Camelia' 'Edredón 2 Plazas Caledonia'
 'Sábanas 300 Hilos - Queen' 'Edredón Marena - 2 Plazas'
 'Set de Edredón Alcione' 'Set de Cubrecama Quilt Lyla'
 'Sábanas 300 Hilos - 1.5 Plazas' 'Edredón Azurium - 1.5 Plazas'
 'Sábanas 300 Hilos - King' 'Edredón Royal - King' 'Edredón Marena - King'
 'Sábanas Algodón 300 Hilos' 'Edredón 2 Plazas Camelia'
 'Edredón 2 Plazas Columbia' 'Edredón King Columbia'
 'Set de Cubrecama Quilt Peach' 'Edredón Dalia - 

In [77]:
# Función para limpiar y generalizar los nombres de los productos
def limpiar_producto(producto):
    producto = producto.lower()  # Convertir a minúsculas para estandarizar
    
    # Generalizar las sábanas de algodón 300 hilos
    if '300 hilos' in producto or '300 hilos' in producto:
        return 'Sábanas Algodón 300 Hilos'
    
    # Generalizar edredones al formato "Edredón Modelo Tamaño"
    if 'edredón' in producto or 'edredon' in producto:
        producto = producto.replace('edredón', 'edredon').replace('-', '').strip()  # Estandarizar "edredón"
        # Dividir el nombre y reordenar para cumplir el formato "Edredón Modelo Tamaño"
        partes = producto.split()
        modelo = partes[1]  # El modelo debería estar en la posición 1
        tamaño = partes[-1]  # El tamaño debería estar al final
        return f'Edredón {modelo.capitalize()} {tamaño.capitalize()}'
    
    # Generalizar cubrecamas al formato "Cubrecama Modelo Tamaño"
    if 'cubrecama' in producto:
        producto = producto.replace('-', '').strip()  # Eliminar guiones
        partes = producto.split()
        modelo = partes[1]  # El modelo debería estar en la posición 1
        tamaño = partes[-1]  # El tamaño debería estar al final
        return f'Cubrecama {modelo.capitalize()} {tamaño.capitalize()}'
    
    # Generalizar fundas de plumón al formato "Funda de Plumón Modelo"
    if 'funda plumón' in producto or 'funda plumon' in producto:
        producto = producto.replace('funda plumón', 'funda de plumon').replace('funda plumon', 'funda de plumon').strip()
        partes = producto.split()
        modelo = partes[-1]  # El modelo debería estar al final
        return f'Funda de Plumón {modelo.capitalize()}'
    
    # Generalizar fundas de almohada al formato "Funda de Almohada Modelo"
    if 'fundas de almohada' in producto or 'funda de almohada' in producto:
        partes = producto.split()
        modelo = partes[-1]  # El modelo debería estar al final
        return f'Funda de Almohada {modelo.capitalize()}'
    
    # Mantener "Plumón Hipoalergénico" tal cual
    if 'plumón hipoalergénico' in producto or 'plumon hipoalergenico' in producto:
        return 'Plumón Hipoalergénico'
    
    if 'prueba' in producto or 'envío' in producto or 'envio' in producto or 'estampado' in producto or 'gift' in producto:
        return 'Otro'
    # Para otros casos, devolver el nombre original sin cambios mayores
    return producto.strip()

# Aplicar la función de limpieza a la columna 'title'
lineitems['title_limpio'] = lineitems['title'].apply(limpiar_producto)

# Ver los valores únicos después de la limpieza
valores_unicos_title_limpio = lineitems['title_limpio'].unique()

# Mostrar los valores limpios únicos
print(valores_unicos_title_limpio)


['Sábanas Algodón 300 Hilos' 'Cubrecama Quilt Microfibra'
 'set de sábanas signature' 'Cubrecama De Spring' 'Edredón De Riansares'
 'Cubrecama De Royce' 'Edredón Royal Queen' 'Edredón De Azurium'
 'Cubrecama 2 Royce' 'Cubrecama 2 Peach' 'Edredón De Caledonia'
 'Edredón De Galdina' 'Edredón Azurium Plazas' 'Edredón De Marena'
 'Edredón 1.5 Galdina' 'Edredón Royal Plazas' 'Edredón De Camelia'
 'Edredón 2 Caledonia' 'Edredón Marena Plazas' 'Edredón De Alcione'
 'Cubrecama De Lyla' 'Edredón Royal King' 'Edredón Marena King'
 'Edredón 2 Camelia' 'Edredón 2 Columbia' 'Edredón King Columbia'
 'Cubrecama De Peach' 'Edredón Dalia Plazas' 'Edredón Queen Camelia'
 'Cubrecama King Peach' 'Edredón Queen Galdina' 'Edredón Queen Finestres'
 'Cubrecama Queen Spring' 'Edredón Queen Queralt' 'Edredón Azurium King'
 'Edredón 2 Azurium' 'Edredón 2 Marena' 'Edredón King Caledonia'
 'Edredón Dalia King' 'Edredón 2 Finestres' 'Cubrecama 2 Spring'
 'Edredón 1.5 Finestres' 'Edredón Azurium Queen' 'Edredón 1.5 

In [81]:
df_orders_clean['hora'] = df_orders_clean['createdAt'].dt.hour
df_orders_clean[['createdAt', 'hora']].head()
df_orders_clean.head()

,order_id,createdAt,currentSubtotalLineItemsQuantity,customer_id,city,address1,address2,hora
0,605504045125,2018-09-11 03:41:34+00:00,0,722083086405,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina,3
1,3823615967295,2021-06-03 00:55:17+00:00,0,722083086405,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina,0
2,5837952581695,2024-07-02 15:24:00+00:00,0,722083086405,Lima,Laa lomas,Unknown,15
3,596940783685,2018-09-04 15:59:16+00:00,0,722885574725,Lima,Av. Las Lomas De La Molina Vieja 486,La Molina,15
4,598392078405,2018-09-05 22:13:29+00:00,0,722986827845,Ciudad 1,Jr Direccion 1,Referencia 1,22


In [88]:
import pandas as pd

df_orders_clean['address_full'] = df_orders_clean['address1'] + ', ' + df_orders_clean['address2']
df_orders_clean['full_address_with_city'] = df_orders_clean['address_full'] + ', ' + df_orders_clean['city']
df_orders_clean[['address1', 'address2', 'city', 'address_full', 'full_address_with_city']].head()


,address1,address2,city,address_full,full_address_with_city
0,Av. Las Lomas De La Molina Vieja 486,La Molina,Lima,"Av. Las Lomas De La Molina Vieja 486, La Molina","Av. Las Lomas De La Molina Vieja 486, La Molin..."
1,Av. Las Lomas De La Molina Vieja 486,La Molina,Lima,"Av. Las Lomas De La Molina Vieja 486, La Molina","Av. Las Lomas De La Molina Vieja 486, La Molin..."
2,Laa lomas,Unknown,Lima,"Laa lomas, Unknown","Laa lomas, Unknown, Lima"
3,Av. Las Lomas De La Molina Vieja 486,La Molina,Lima,"Av. Las Lomas De La Molina Vieja 486, La Molina","Av. Las Lomas De La Molina Vieja 486, La Molin..."
4,Jr Direccion 1,Referencia 1,Ciudad 1,"Jr Direccion 1, Referencia 1","Jr Direccion 1, Referencia 1, Ciudad 1"


In [92]:
# Ver los valores únicos en la columna 'source' del DataFrame customerJourney
valores_unicos_source1 = df_orders_clean['city'].unique()

# Mostrar los valores
print(valores_unicos_source1)

['Lima' 'Ciudad 1' 'Unknown' 'Cuadra 4 de Pedro Ventura' 'Callao' 'Pisco'
 'LIMA' 'Chiclayo' 'Ayacucho' 'Jesús De Nazareno' 'Mollendo' 'Talara'
 'Chimbote' 'Lima ' 'lima' 'Villa el Salvador'
 'Frente a mega plaza y al costado de una posta de esalud'
 'Por el óvalo Independencia de Sucre' 'espalda del colegio Jean Piaget'
 'Huacho' 'Arica' 'Urb. Virgen del Carmen' 'Detras capitañia del puerto'
 'Pimentel' 'A una cuadra de la I E Divino Niño' 'Paradero 2 las flores'
 'Espalda del mercado modelo en 28 de julio, altura del ministerio de aeronautica'
 'Costado del Ipcna cusco' '3 pisos color verde porton negro' 'San Martín'
 'A espalda del Las turquesas ( antiguo Inabif)'
 'Por la. Av dolores en la. Pizzeria presto bajar una cuadra '
 'Galeria adderly a 1 cuadra de parque canepa' 'Cercado '
 'Alt. Cuadra 15 av. Argentina'
 'Al lado del instituto itep (av. Alfonso ugarte)' 'TRUJILLO' 'Tacna '
 'Moquegua, Mcal Nieto, Moquegua ' 'Tacna' 'Juliaca'
 'Entre los postes y Jorge Basadre'
 ' Alt 9 Av

In [93]:
df_orders_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   order_id                          3813 non-null   object             
 1   createdAt                         3813 non-null   datetime64[ns, UTC]
 2   currentSubtotalLineItemsQuantity  3813 non-null   int64              
 3   customer_id                       3813 non-null   object             
 4   city                              3813 non-null   object             
 5   address1                          3813 non-null   object             
 6   address2                          3813 non-null   object             
 7   hora                              3813 non-null   int32              
 8   address_full                      3813 non-null   object             
 9   full_address_with_city            3813 non-null   object       

In [87]:
# Ver los valores únicos en la columna 'source' del DataFrame customerJourney
valores_unicos_source = df_orders_clean['city'].unique()

# Mostrar los valores
print(valores_unicos_source)


['Lima' 'Ciudad 1' 'Unknown' 'Cuadra 4 de Pedro Ventura' 'Callao' 'Pisco'
 'LIMA' 'Chiclayo' 'Ayacucho' 'Jesús De Nazareno' 'Mollendo' 'Talara'
 'Chimbote' 'Lima ' 'lima' 'Villa el Salvador'
 'Frente a mega plaza y al costado de una posta de esalud'
 'Por el óvalo Independencia de Sucre' 'espalda del colegio Jean Piaget'
 'Huacho' 'Arica' 'Urb. Virgen del Carmen' 'Detras capitañia del puerto'
 'Pimentel' 'A una cuadra de la I E Divino Niño' 'Paradero 2 las flores'
 'Espalda del mercado modelo en 28 de julio, altura del ministerio de aeronautica'
 'Costado del Ipcna cusco' '3 pisos color verde porton negro' 'San Martín'
 'A espalda del Las turquesas ( antiguo Inabif)'
 'Por la. Av dolores en la. Pizzeria presto bajar una cuadra '
 'Galeria adderly a 1 cuadra de parque canepa' 'Cercado '
 'Alt. Cuadra 15 av. Argentina'
 'Al lado del instituto itep (av. Alfonso ugarte)' 'TRUJILLO' 'Tacna '
 'Moquegua, Mcal Nieto, Moquegua ' 'Tacna' 'Juliaca'
 'Entre los postes y Jorge Basadre'
 ' Alt 9 Av

In [106]:
import re

# Diccionario con distritos de Lima y sus correspondientes códigos postales
distritos_codigos_postales = {
    'san isidro': 'San Isidro',
    'miraflores': 'Miraflores',
    'surco': 'Surco',
    'san borja': 'San Borja',
    'la molina': 'La Molina',
    'lince': 'Lince',
    'magdalena del mar': 'Magdalena del Mar',
    'pueblo libre': 'Pueblo libre',
    'jesús maría': 'Jesus María',
    'rimac': 'Rímac',
    'san miguel': 'San Miguel',
    'ate': 'Ate',
    'barranco': 'Barranco',
    'villa el salvador': 'Villa el Salvador',
    'villa maria del triunfo': 'Villa Maria del Triunfo',
    'chorrillos': 'Chorrillos',
    'breña': 'Breña',
    'san juan de lurigancho': 'San Juan de Lurigancho',
    'san juan de miraflores': 'San Juan de Miraflores',
    'comas': 'Comas',
    'independencia': 'Independencia',
    'los olivos': 'Los Olivos',
    'carabayllo': 'Carabayllo',
    'san martín de porres': 'San Martin de Porres',
    'la victoria': 'La Victoria',
    'cercado de lima': 'Cercador de Lima',
    'el agustino': 'El Agustino',
    'surquillo': 'Surquillo',
    'santa anita': 'Santa Anita',
    'puente piedra': 'Puente Piedra',
    'san luis': 'San Luis',
    'lurigancho': 'Lurigancho',
    'lima': 'Cercado de Lima',
    'desconocido': 'Cercado de Lima',
    'Unknown': 'Cercado de Lima'
}

# Diccionario con códigos postales de algunas provincias principales
provincias_codigos_postales = {
    'arequipa': 'Arequipa',
    'ayacucho': 'Ayacucho',
    'trujillo': 'Trujillo',
    'piura': 'Piura',
    'ica': 'Ica',
    'chiclayo': 'Chiclayo',
    'cajamarca': 'Cajamarca',
    'cusco': 'Cusco',
    'huánuco': 'Huánuco',
    'moquegua': 'Moquegua',
    'tacna': 'Tacnca',
    'puno': 'Puno',
    'talara': 'Talara',
    'huacho': 'Huacho',
    'huaraz': 'Huaraz',
    'chimbote': 'Chimbote',
    'moyobamba': 'Moyobamba',
    'nasca': 'Nasca',
    'pucallpa': 'Pucallpa',
    'callao': 'Callao',
    'pisco': 'Pisco',
    'mollendo': 'Mollendo',
    'nazareno': 'Nazareno'
    # Añadir más si es necesario
}

# Función para estandarizar las referencias o distritos conocidos a su código postal
def estandarizar_referencias_o_distritos(direccion):
    direccion = direccion.lower().strip()

    # 1. Revisar si coincide con un distrito de Lima
    for distrito, codigo_postal in distritos_codigos_postales.items():
        if re.search(distrito, direccion):
            return codigo_postal
    
    # 2. Revisar si coincide con una provincia fuera de Lima
    for provincia, codigo_postal in provincias_codigos_postales.items():
        if re.search(provincia, direccion):
            return codigo_postal
    
    # 3. Revisar las referencias comunes y asignar código postal general para Lima
    referencias_comunes = [
        'frente al estadio', 'óvalo independencia', 'espalda del colegio', 
        'paradero', 'costado', 'altura', 'plaza', 'cuadra', 'por el óvalo', 
        'residencial', 'edificio', 'por la puerta', 'por la av', 'altura cuadra', 'ciudad'
    ]
    
    for referencia in referencias_comunes:
        if re.search(referencia, direccion):
            return 'Cercado de Lima'  # Código postal general para Lima

    # 4. Si no coincide, devolver 'Desconocido'
    return 'Cercado de Lima'

# Aplicar la función a la columna 'city' del DataFrame
df_orders_clean['clean_city'] = df_orders_clean['city'].apply(estandarizar_referencias_o_distritos)

# Ver los resultados
df_orders_clean[['city', 'clean_city']].head(50)


,city,clean_city
0,Lima,Cercado de Lima
1,Lima,Cercado de Lima
2,Lima,Cercado de Lima
3,Lima,Cercado de Lima
4,Ciudad 1,Cercado de Lima
5,Ciudad 1,Cercado de Lima
6,Lima,Cercado de Lima
7,Unknown,Cercado de Lima
8,Unknown,Cercado de Lima
9,Lima,Cercado de Lima


In [107]:
# Ver los valores únicos en la columna 'source' del DataFrame customerJourney
valores_unicos_source = df_orders_clean['maps'].unique()

# Mostrar los valores
print(valores_unicos_source)

['Cercado de Lima' 'Callao' 'Pisco' 'Chiclayo' 'Ayacucho' 'Nazareno'
 'Mollendo' 'Talara' 'Chimbote' 'Villa el Salvador' 'Independencia'
 'Huacho' 'Ica' 'Cusco' 'Trujillo' 'Tacnca' 'Moquegua' 'La Molina'
 'Chorrillos' 'Cajamarca' 'Pueblo libre' 'Breña' 'Miraflores' 'Surco'
 'Rímac' 'San Luis' 'Carabayllo' 'San Isidro' 'Comas' 'San Borja' 'Ate'
 'Jesus María' 'La Victoria' 'Magdalena del Mar' 'Arequipa'
 'Cercador de Lima' 'Surquillo' 'Puente Piedra' 'Los Olivos' 'San Miguel'
 'Moyobamba' 'Santa Anita' 'El Agustino' 'Villa Maria del Triunfo'
 'San Juan de Lurigancho' 'San Martin de Porres' 'Lince' 'Barranco'
 'Piura' 'Huaraz' 'Lurigancho' 'Puno' 'Huánuco' 'Nasca']


In [109]:
import pandas as pd

# Unir address1 y address2 con coma y espacio
df_orders_clean['maps'] = df_orders_clean['clean_city'] + ', Perú' 


# Ver el resultado
df_orders_clean[['maps']].head()

,maps
0,"Cercado de Lima, Perú"
1,"Cercado de Lima, Perú"
2,"Cercado de Lima, Perú"
3,"Cercado de Lima, Perú"
4,"Cercado de Lima, Perú"


In [115]:
df_orders_clean

,order_id,createdAt,currentSubtotalLineItemsQuantity,customer_id,city,hora,maps,clean_city
0,605504045125,2018-09-11 03:41:34+00:00,0,722083086405,Lima,3,"Cercado de Lima, Perú",Cercado de Lima
1,3823615967295,2021-06-03 00:55:17+00:00,0,722083086405,Lima,0,"Cercado de Lima, Perú",Cercado de Lima
2,5837952581695,2024-07-02 15:24:00+00:00,0,722083086405,Lima,15,"Cercado de Lima, Perú",Cercado de Lima
3,596940783685,2018-09-04 15:59:16+00:00,0,722885574725,Lima,15,"Cercado de Lima, Perú",Cercado de Lima
4,598392078405,2018-09-05 22:13:29+00:00,0,722986827845,Ciudad 1,22,"Cercado de Lima, Perú",Cercado de Lima
...,...,...,...,...,...,...,...,...
3808,5858791981119,2024-07-15 18:30:04+00:00,1,7259410300991,Lima,18,"Cercado de Lima, Perú",Cercado de Lima
3809,5902652735551,2024-08-08 18:29:27+00:00,1,7344207790143,Lince,18,"Lince, Perú",Lince
3810,5905901879359,2024-08-10 18:51:15+00:00,1,7346496929855,Lima,18,"Cercado de Lima, Perú",Cercado de Lima
3811,5966086537279,2024-09-10 04:11:57+00:00,1,7399752794175,Lima,4,"Cercado de Lima, Perú",Cercado de Lima


In [117]:
df_lineitems_clean = lineitems.drop(columns=['title'])
df_lineitems_clean = df_lineitems_clean.drop(columns=['product'])
df_lineitems_clean = df_lineitems_clean.drop(columns=['originalUnitPriceSet'])
df_lineitems_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5260 entries, 0 to 5259
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lineitem_id   5260 non-null   object 
 1   quantity      5260 non-null   int64  
 2   order_id      5260 non-null   object 
 3   price         5260 non-null   float64
 4   title_limpio  5260 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 205.6+ KB


In [118]:
import pandas as pd

#Calcular el precio total para cada línea en df_lineitems_clean
# Multiplicar la cantidad por el precio
df_lineitems_clean['line_total'] = df_lineitems_clean['quantity'] * df_lineitems_clean['price']

# Agrupar por order_id para obtener el total de cada orden
df_totals = df_lineitems_clean.groupby('order_id', as_index=False)['line_total'].sum()

# Renombrar la columna para que sea más clara
df_totals.rename(columns={'line_total': 'order_total'}, inplace=True)

# Unir los totales con df_orders_clean en función del order_id
df_orders_clean = pd.merge(df_orders_clean, df_totals, on='order_id', how='left')

# Verificar el resultado
df_orders_clean[['order_id', 'order_total']].head()


,order_id,order_total
0,605504045125,149.0
1,3823615967295,184.0
2,5837952581695,392.0
3,596940783685,519.5
4,598392078405,429.7


In [120]:
df_orders_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   order_id                          3813 non-null   object             
 1   createdAt                         3813 non-null   datetime64[ns, UTC]
 2   currentSubtotalLineItemsQuantity  3813 non-null   int64              
 3   customer_id                       3813 non-null   object             
 4   city                              3813 non-null   object             
 5   hora                              3813 non-null   int32              
 6   maps                              3813 non-null   object             
 7   clean_city                        3813 non-null   object             
 8   order_total                       3813 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), int32(1), int64(1), objec

In [121]:
df_customerJourney_clean = df_customerJourney.drop(columns=['source'])
df_customerJourney_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   firstVisit_id        2749 non-null   object
 1   order_id             3813 non-null   object
 2   source_generalizada  3813 non-null   object
dtypes: object(3)
memory usage: 89.5+ KB


In [122]:
df_customerJourney_clean.head()

,firstVisit_id,order_id,source_generalizada
0,NaN,605504045125,Unknown
1,NaN,3823615967295,Unknown
2,13755680587839,5837952581695,direct
3,NaN,596940783685,Unknown
4,NaN,598392078405,Unknown


In [123]:
# Exportar cada DataFrame como un archivo CSV
customers.to_csv('customers.csv', index=False)
df_orders_clean.to_csv('orders.csv', index=False)
df_lineitems_clean.to_csv('lineitems.csv', index=False)
df_customerJourney_clean.to_csv('customerJourney.csv', index=False)
